In [23]:
"""
adapted from 
https://github.com/pq-crystals/security-estimates/blob/master/proba_util.py

"""


def binomial(x, y):
    """ Binomial coefficient
    :param x: (integer)
    :param y: (integer)
    :returns: y choose x
    """
    try:
        binom = factorial(x) // factorial(y) // factorial(x - y)
    except ValueError:
        binom = 0
    return binom


def centered_binomial_pdf(k, x):
    """ Probability density function of the centered binomial law of param k at x
    :param k: (integer)
    :param x: (integer)
    :returns: p_k(x)
    """
    return binomial(2 * k, x + k) / 2. ** (2 * k)


def build_centered_binomial_law(k):
    """ Construct the binomial law as a dictionnary
    :param k: (integer)
    :param x: (integer)
    :returns: A dictionnary {x:p_k(x) for x in {-k..k}}
    """
    D = {}
    for i in range(-k, k + 1):
        D[i] = centered_binomial_pdf(k, i)
    return D


def law_convolution(A, B):
    """ Construct the convolution of two laws (sum of independent variables from two input laws)
    :param A: first input law (dictionnary)
    :param B: second input law (dictionnary)
    """

    C = {}
    for a in A:
        for b in B:
            c = a + b
            C[c] = C.get(c, 0) + A[a] * B[b]
    return C

In [24]:

"""
SIS part function
"""
def cardinality_s(alpha, n):  # |S_alpha|
    return (2 * alpha + 1) ** n  # n is the number of coefficient. eg. f0 ~ f255

def calculate_volume(radius, n):  # V_n(R)
    return (((math.pi / 2) ** int(n / 2)) / n.multifactorial(2)) * ((2 * radius) ** n)

def SIS_tau_partial(q, l, k, alpha1, alpha2, n, wi_inf, wi):
    S_alpha1_wi = cardinality_s(alpha1 + wi_inf, n)
    S_alpha2_wi = cardinality_s(alpha2 + wi_inf, n)
    up = (S_alpha1_wi ** l) * (S_alpha2_wi ** k)
    down = (wi ** (l + k))
    return up / down

"""
display convinience
"""
def lg2_convert(number):
    return RR(log(number, 2))



In [25]:
# def para_set
class Params:
    def __init__(self, n, k, q, eta1, eta2, du, dv, total_space):
        self.n = n
        self.k = k
        self.q = q
        self.eta1 = eta1
        self.eta2 = eta2
        self.du = du
        self.dv = dv
        self.total_space = total_space

In [26]:

# CBD part
def CBD_part(para_set):
    # paras
    n, k, q, eta1, eta2, du, dv, total_space = para_set.n, para_set.k, para_set.q, para_set.eta1, para_set.eta2, \
                                               para_set.du, para_set.dv, para_set.total_space

    # cbd for single coefficient in delta r & e1
    dict_r = build_centered_binomial_law(eta1)
    dict_e1 = build_centered_binomial_law(eta2)
    delta_r_single = law_convolution(dict_r, dict_r)
    delta_e1_single = law_convolution(dict_e1, dict_e1)

    # prob delta r
    prob_delta_r = delta_r_single[0] ** (n * k)

    # prob delta e1 in [-floor(eu), floor(eu)]
    eu = q // (2 ** du)
    prob_e1_single = 0
    for eu_term in range(-1 * eu, eu + 1):
        prob_e1_single += delta_e1_single[eu_term]
    prob_delta_e1 = prob_e1_single ** (n * k)

    # produce delta r + delta e1 in CBD
    prob_CBD = prob_delta_r * prob_delta_e1

    print("prob_CBD without total space in lg2: ", lg2_convert(prob_CBD))
    print("prob_CBD without total space in lg2 sqrt: ", lg2_convert(prob_CBD) / 2)
    print("prob_CBD with total space in lg2: ", lg2_convert(prob_CBD * total_space))
    print("prob_CBD with total space in lg2 sqrt: ", lg2_convert(prob_CBD * total_space) / 2)
    
    return prob_CBD

In [27]:
# SIS prob

def SIS_s(q, l, k, alpha1, alpha2, n):
    S_alpha1 = cardinality_s(alpha1, n)
    S_alpha2 = cardinality_s(alpha2, n)
    up = (S_alpha1 ** l) * (S_alpha2 ** k)
    down = q ** (n * k)
    return up / down


def SIS_tau(q, l, k, alpha1, alpha2, n, d):
    e = int(d * log(alpha1 * sqrt(n), q))
    print("e: ", e)
    result = 0
    single_list = []
    result_list = []

    for i in range(1, e + 1):
        # fix part
        t = int(q ** (i / d) / 2)
        fix_part = binomial(d, i) / (q ** (n * k * (1 - i / d)))

        # wi comparision
        if i == 1:
            W_i_inf = 1  # ||W||_inf
            W_i_cardinality = 2 * n  # |W1| = 2n
            partial = SIS_tau_partial(q, l, k, alpha1, alpha2, n, W_i_inf, W_i_cardinality)
        elif t < sqrt(n):
            W_i_inf = 1
            W_i_cardinality = 0
            for j in range(t ** 2 + 1):
                W_i_cardinality += binomial(n, j) * (2 ** j)
            partial = SIS_tau_partial(q, l, k, alpha1, alpha2, n, W_i_inf, W_i_cardinality)
        else:
            # comparision

            set1_W_i_inf = t
            radius = q ** (i / d) / 2 - sqrt(n)
            set1_W_i_cardinality = RR(calculate_volume(radius, n))

            set2_W_i_inf = int(t / sqrt(n))
            set2_W_i_cardinality = (2 * set2_W_i_inf + 1) ** n

            compare_1 = SIS_tau_partial(q, l, k, alpha1, alpha2, n, set1_W_i_inf, set1_W_i_cardinality)
            compare_2 = SIS_tau_partial(q, l, k, alpha1, alpha2, n, set2_W_i_inf, set2_W_i_cardinality)
            partial = min(compare_1, compare_2)
#             if (compare_2 > compare_1):
#                 print("outlier round: ", i)

        single_round = fix_part * partial
        single_list.append(lg2_convert(single_round))
        result += single_round
        result_list.append(lg2_convert(result))
        
    result_plot = []
    result_plot.append(result)
    result_plot.append(single_list)
    result_plot.append(result_list)
    
    return result_plot
    
    
    #return result


def SIS_part(para_set):
    # paras
    n, k, q, eta1, eta2, du, dv, total_space = para_set.n, para_set.k, para_set.q, para_set.eta1, para_set.eta2, \
                                               para_set.du, para_set.dv, para_set.total_space
    
    d = 128
    eu = int(q // (2 ** du))
    alpha1 = 2 * eta1
    alpha2 = 2 * eta2 + eu
    s_part = SIS_s(q, k, k, alpha1, alpha2, n)
    tau_part = SIS_tau(q, k, k, alpha1, alpha2, n, d)[0]
    result = s_part + tau_part

    # times delta r != 0
    dict_r = build_centered_binomial_law(eta1)
    delta_r_single = law_convolution(dict_r, dict_r)
    prob_delta_r = delta_r_single[0] ** (n * k)
    prob_r_non_zero = 1 # close enough to 1

    result = prob_r_non_zero * result
    cumulative_result = total_space * result
    print("SIS without total space in lg2: ", lg2_convert(result))
    print("SIS without total space in lg2 sqrt: ", lg2_convert(result) / 2)
    print("SIS with total space in lg2: ", lg2_convert(cumulative_result))
    print("SIS with total space in lg2 sqrt: ", lg2_convert(cumulative_result) / 2)
#     print("---")
    
#     single_list = SIS_tau(q, k, k, alpha1, alpha2, n, d)[1]
#     result_list = SIS_tau(q, k, k, alpha1, alpha2, n, d)[2]
#     plot_single = list_plot([single_list[i] for i in range(len(single_list))], title="Single round for set 3", axes_labels=[r'Round', r'Probability(in log2)'])
#     plot_result = list_plot([result_list[i] for i in range(len(result_list))], title="Cumulative result for set 3", axes_labels=[r'Round', r'Probability(in log2)'])
#     # axes_labels=[r'Round', r'Probability(in log2)']
#     plot_single.show()
#     plot_result.show()
            
    return result


In [28]:
# total injectivity

def total_injectivity(para_set):
    CBD_prob = CBD_part(para_set)
    print("---")
    SIS_prob = SIS_part(para_set)
    print("---")

    total_space = para_set.total_space
    total_prob = total_space * CBD_prob + SIS_prob
    print("total_prob in lg2 with message space: ", lg2_convert(total_prob))
    print("total_prob in lg2 sqrt with message space: ", lg2_convert(total_prob) / 2)

In [29]:
# parameter sets

# set = n, k, q, eta1, eta2, du, dv, total message space

total_space = 2 ** 511  # binomial (2**256,2)
set1 = Params(256, 2, 3329, 3, 2, 10, 4, total_space)
set2 = Params(256, 3, 3329, 2, 2, 10, 4, total_space)
set3 = Params(256, 4, 3329, 2, 2, 11, 5, total_space)

print("+++++++++++++++++")
print("+++ Para Set1 +++")
print("+++++++++++++++++")
total_injectivity(set1)
print("\n")
print("+++++++++++++++++")
print("+++ Para Set2 +++")
print("+++++++++++++++++")
total_injectivity(set2)
print("\n")
print("+++++++++++++++++")
print("+++ Para Set3 +++")
print("+++++++++++++++++")
total_injectivity(set3)

+++++++++++++++++
+++ Para Set1 +++
+++++++++++++++++
prob_CBD without total space in lg2:  -1105.69793116763
prob_CBD without total space in lg2 sqrt:  -552.848965583815
prob_CBD with total space in lg2:  -594.697931167630
prob_CBD with total space in lg2 sqrt:  -297.348965583815
---
e:  72
SIS without total space in lg2:  -354.363609774930
SIS without total space in lg2 sqrt:  -177.181804887465
SIS with total space in lg2:  156.636390225070
SIS with total space in lg2 sqrt:  78.3181951125352
---
total_prob in lg2 with message space:  -354.363609774930
total_prob in lg2 sqrt with message space:  -177.181804887465


+++++++++++++++++
+++ Para Set2 +++
+++++++++++++++++
prob_CBD without total space in lg2:  -1445.40080354524
prob_CBD without total space in lg2 sqrt:  -722.700401772621
prob_CBD with total space in lg2:  -934.400803545242
prob_CBD with total space in lg2 sqrt:  -467.200401772621
---
e:  65
SIS without total space in lg2:  -937.268442169865
SIS without total space in lg2 s